In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

df = pd.read_csv("Sentiment_Analysis.txt")
df.head()

,article_id,article_text,source
0,1,"If you stop and think about it, you can probab...",https://study.com/academy/lesson/animal-body-p...
1,2,It all depends on what’s in the atmosphere! Fo...,https://spaceplace.nasa.gov/blue-sky/en/
2,3,A lot of the Australian middle order will be s...,https://www.abc.net.au/news/2021-01-16/austral...


In [3]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [4]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

C:\Users\admin\Anaconda3\envs\R-TUTT\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [5]:
sentence_vectors = []
word_embeddings = {}
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [6]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
              sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [7]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [8]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(3):
    print("ARTICLE:")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')

ARTICLE:
If you stop and think about it, you can probably come up with a long list of the different types of animals found on Earth. But there are so many more than that! I bet there are some that you have never even heard of or didn't even know were considered animals.The diversity of animal life on Earth is indeed quite vast. Fortunately, there are ways to make sense of it. We do this by classifying animals based on certain body structures as well as how those body structures are combined to make a whole animal.Together the organization and combination of body structures describes an animal's body plan. For example, you have legs, which is one of your body structures. But you are quite different from a crayfish, which also has legs! Our overall combination of body structures - our body plan - is not even close to that of a crayfish, even though we share some of the same structures. And this combination and organization of structures is how we differentiate between different types of 